## One Node Training

In [10]:
# Importing Librairies
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, InputLayer, Reshape, Rescaling
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.utils import to_categorical
from keras.regularizers import l2
import matplotlib.pyplot as plt
import numpy as np
import os

# Uncomment next line to : Force using CPU only by making GPU invisible
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [30]:
# Importing & Preprocessing CIFAR-10 Dataset from Directory
img_height = 32
img_width = 32
trainval_path = 'C:/Users/PC/Desktop/DATASET/CIFAR-10/'
test_path = ''

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict


def load_dataset():
    # Loading Training Dataset
    x_train = 0
    y_train = 0
    i = 1    
    while i <= 5:
        filename = 'data_batch_' + str(i)
        fullpath = os.path.join(trainval_path, filename)
        data_batch = unpickle(fullpath)
        x = data_batch[b'data']
        y = data_batch[b'labels']
        x = x.reshape(len(x), 3, img_height, img_width).transpose(0, 2, 3, 1)
        x = x/255.0
        if i==1:
            x_train = x
            y_train = y
        else:
            x_train = np.concatenate((x_train, x))
            y_train = np.concatenate((y_train, y))
        i = i + 1
    
    # Loading Test/Validation Dataset
    fullpath = os.path.join(trainval_path, 'test_batch')
    data_batch = unpickle(fullpath)
    x_test = data_batch[b'data']
    x_test = x_test.reshape(len(x_test), 3, img_height, img_width).transpose(0, 2, 3, 1)
    y_test = data_batch[b'labels']
    y_test = np.asarray(y_test)
    
    # Normalizing Images
    x_train, x_test = normalize_images(x_train, x_test)
    # Convert to Categorical
    return (x_train, y_train), (x_test, y_test)



def normalize_images(train, test):
    # convert 0-255 integers to floats
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # scaling values from 0-255 to 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    # return normalized images
    return train_norm, test_norm



'''TEST IF DATASET IS LOADED CORRECTLY'''
'''
(x_train, y_train), (x_test, y_test) = load_dataset()
print(x_test.shape)
plt.imshow(x_train[0])
plt.show()
'''

'\n(x_train, y_train), (x_test, y_test) = load_dataset()\nprint(x_test.shape)\nplt.imshow(x_train[0])\nplt.show()\n'

In [28]:
# Create and Compile Model
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(32, 32, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.3))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

model.compile(optimizer=optimizer, loss=loss_fn, metrics=[accuracy])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 32, 32, 64)        1792      
                                                                 
 batch_normalization_36 (Bat  (None, 32, 32, 64)       256       
 chNormalization)                                                
                                                                 
 conv2d_29 (Conv2D)          (None, 32, 32, 64)        36928     
                                                                 
 batch_normalization_37 (Bat  (None, 32, 32, 64)       256       
 chNormalization)                                                
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 16, 16, 64)       0         
 g2D)                                                            
                                                      

In [ ]:
# Create Topless Pre-trained VGG16 Model on ImageNet
model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Adding Classifier Layers
x = model.output
fl = Flatten()(x)
fc1 = Dense(1024, activation='relu')(fl)
out = Dense(20, activation='softmax')(fc1)
# Concatenate Model
model = Model(inputs=model.input, outputs=out)

# Let all layers trainable (Fine tuning Total)
for layer in model.layers[:-2]:
   layer.trainable = False

model.compile(optimizer='adam', loss='sparse_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# callbacks - variable learning rate
initial_learning_rate = 0.01
lr_schedule_cb = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=20, decay_rate=0.96, staircase=True)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "model.h5", save_best_only=True)

# callback - early stopping
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True)

In [ ]:
# Training and Evaluating Model
# load CIFAR-10 Dataset
(x_train, y_train), (x_test, y_test) = load_dataset()
# Data Augmentation Generator
datagen = ImageDataGenerator(
            featurewise_center=True,
            featurewise_std_normalization=True,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True,
            validation_split=0.2)
datagen.fit(x_train)
# Data iterator
iter_train = datagen.flow(x_train, y_train, batch_size=64)
# Calculating Steps par Epoch
steps = int(len(x_train) / 64)
# Training the model
history = model.fit(iter_train, validation_data=(x_test, y_test), steps_per_epoch=steps, epochs=100)
score = model.evaluate(x_test, y_test)

Epoch 1/100
781/781 [==============================] - 27s 31ms/step - loss: 1.9766 - sparse_categorical_accuracy: 0.3465 - val_loss: 2.9171 - val_sparse_categorical_accuracy: 0.1704
Epoch 2/100
781/781 [==============================] - 24s 31ms/step - loss: 1.3551 - sparse_categorical_accuracy: 0.5125 - val_loss: 2.7909 - val_sparse_categorical_accuracy: 0.2524
Epoch 3/100
781/781 [==============================] - 24s 31ms/step - loss: 1.1272 - sparse_categorical_accuracy: 0.6053 - val_loss: 2.4255 - val_sparse_categorical_accuracy: 0.3135
Epoch 4/100
781/781 [==============================] - 25s 32ms/step - loss: 0.9757 - sparse_categorical_accuracy: 0.6607 - val_loss: 2.0054 - val_sparse_categorical_accuracy: 0.3819
Epoch 5/100
781/781 [==============================] - 25s 32ms/step - loss: 0.8755 - sparse_categorical_accuracy: 0.6989 - val_loss: 2.6461 - val_sparse_categorical_accuracy: 0.3100
Epoch 6/100
781/781 [==============================] - 25s 31ms/step - loss: 0.7972 -

In [ ]:
# plot training and validation accuracy
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()
# plot training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

In [ ]:
import tensorflow as tf
print(tf.__version__)